## sub-task 2 제출 스크립트
ETRI FASHION-HOW Season 5 task2 제출코드입니다.
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.
- 작성하신 추론용 코드(예: test.py)를 본 스크립트 내의 etri_task2_submit() 함수로 작동되게끔 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
코드는 크게 4가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치
2. etri_task2_submit() 함수 편집 (추론 스크립트)
3. submit() 함수로 wrapping
4. if __name__ == "__main__" 조건문을 이용한 제출 수행

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

### 2. etri_task2_submit() 함수 편집 (추론 스크립트)
#### etri_task2_submit() 편집 시 주의사항

1. 아래 etri_task2_submit() 함수 내에 전체 추론 실행 코드를 삽입하고 결과를 dataframe으로 return하게끔 구성
   - Baseline이 아닌 다른 모델을 사용하는 경우에도 동일
2. 함수 내에서는 import * 가 적용되지 않으므로 필요한 import object를 직접 입력
   - 반드시 함수 내에서 import가 이루어져야 합니다.
3. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
4. 모델 내부의 경로는 ./ 으로 경로를 지정합니다. (예: weight 파일 경로 = ./model/...)
5. 데이터는 **/aif/Dataset/** 경로 아래에 있습니다. (코드 내용 참조) 
6. return할 결과물과 양식에 유의합니다.

In [2]:
def etri_task2_submit():

    from dataset import ETRIDataset_color
    from networks import efficientformer_color

    import pandas as pd
    import numpy as np
    from sklearn.metrics import confusion_matrix

    import torch
    import torch.utils.data
    import torch.utils.data.distributed

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    """ The main function of the test process for performance measurement. """
    net = efficientformer_color().to(DEVICE)
    trained_weights = torch.load('./model/model_11.pt',map_location=DEVICE) # 자기 모델 경로를 지정합니다
    net.load_state_dict(trained_weights)
    net.eval()

    df = pd.read_csv('/aif/Dataset/Fashion-How24_sub2_test.csv') # 제출 시 데이터 경로 준수. /aif/ 아래에 있습니다.
    val_dataset = ETRIDataset_color(df, base_path='/aif/Dataset/test/', mode='val') # 제출 시 데이터 경로 준수. /aif/ 아래에 있습니다.
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0) # 반드시 shuffle=False

    pred_list = np.array([])

    for j, sample in enumerate(val_dataloader):
        for key in sample:
            sample[key] = sample[key].to(DEVICE)
        out = net(sample)

        _, indx = out.max(1)
        pred_list = np.concatenate([pred_list, indx.cpu()], axis=0)
        
    # 예측 결과를 dataframe으로 변환한 다음 함수의 결과로 return합니다.
    # 'image_name', 'color'의 컬럼명과 image_name의 샘플 순서를 지켜주시기 바랍니다.
    # Baseline이 아닌 다른 모델을 사용하는 경우에도 같은 형식의 dataframe으로 return할 수 있도록 합니다.
    out = pd.DataFrame({'image_name':df['image_name'],'color':pred_list})
   
    return out # 반드시 추론결과를 return

### 3. submit() 함수로 wrapping
#### 반드시 아래와 같이 submit() 이라는 함수로 위에 정의된 etri_task2_submit 함수를 wrapping해야 합니다.

In [3]:
def submit():
    return etri_task2_submit

### 4. if name == "main" 조건문을 이용한 제출 수행
#### 아래와 같이 if __name__ == "__main__" 구문 내에서 제출 함수가 실행되게끔 합니다.
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 할당된 계정을 이용하여 task별로 확인하실 수 있습니다.

In [4]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="etri-task2_test",               # 본인의 모델명 입력(버전 관리에 용이하게끔 편의에 맞게 지정합니다)
               key="5024d7d1-807b-4c1c-903b-fd250bd38098",                                  # 본인의 task key 입력
               func=submit                            # 3.에서 wrapping한 submit function
               )
    #-----------------------------------------------------#
    print(time.time() - t)

file : task.py
python
파일 전송 완료
score = 0.8788888889(
325.8901889324188
